Name : Ng Chen Ting 
# Part B
## Task 1. Processing Data Stream
### Event Producer 2

In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import pandas
import csv
import random
import datetime as dt

hostip = "192.168.1.14" 

def load_data_from_csv():
    """
    Load data from 'hotspot_AQUA_streaming.csv' and format it into a list of dictionaries.
    """
    aqua_streaming_data = pandas.read_csv('hotspot_AQUA_streaming.csv') 
    data = []
    
    # Iterate through each row 
    for _, row in aqua_streaming_data.iterrows(): 
        data_row = {} 
        data_row['latitude'] = float(row['latitude'])
        data_row['longitude'] = float(row['longitude'])
        data_row['confidence'] = int(row['confidence'])
        data_row['surface_temperature_celcius'] = int(row['surface_temperature_celcius'])
        data.append(data_row)
    return data

def publish_message(producer_instance, topic_name, value):
    try:
        # To convert them to bytes 
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, value=value_bytes)

        # To ensure the message is actually sent to Kafka 
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(value))
        
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
        
    finally:
        return _producer
    
    
if __name__ == '__main__':
    topic = 'Assignment_TaskB_Hotspots'
    print('Publishing records..')
    producer = connect_kafka_producer()
    data = load_data_from_csv()
    
    while True:
        try:
            # Get a random line from 'hotspot_AQUA_streaming.csv'
            random_line = random.choice(data)

            # Add producer_id to indicate it's from AQUA producer
            random_line['producer_id'] = 'producer_hotspot_aqua'  
            
            # Generate a random time 
            random_time = dt.time(hour=random.randint(0, 23), minute=random.randint(0, 59), second=random.randint(0, 59))
            random_line['created_time'] = random_time.strftime("%H:%M:%S") 

            # Convert the dictionary to a JSON string and publish
            json_data = dumps(random_line) 
            publish_message(producer, topic, json_data) 
            sleep(random.randint(3,7))      # Wait a random time between 1 to 7 seconds

        except KeyboardInterrupt:
            print('Exiting...')
            break

Publishing records..
Message published successfully. Data: {"latitude": -37.333, "longitude": 143.534, "confidence": 88, "surface_temperature_celcius": 58, "producer_id": "producer_hotspot_aqua", "created_time": "16:52:58"}
Message published successfully. Data: {"latitude": -36.9345, "longitude": 143.7675, "confidence": 87, "surface_temperature_celcius": 61, "producer_id": "producer_hotspot_aqua", "created_time": "19:56:21"}
Message published successfully. Data: {"latitude": -36.3492, "longitude": 140.9727, "confidence": 88, "surface_temperature_celcius": 63, "producer_id": "producer_hotspot_aqua", "created_time": "11:59:56"}
Message published successfully. Data: {"latitude": -37.3655, "longitude": 148.2821, "confidence": 85, "surface_temperature_celcius": 59, "producer_id": "producer_hotspot_aqua", "created_time": "21:51:02"}
Message published successfully. Data: {"latitude": -36.4612, "longitude": 144.7775, "confidence": 91, "surface_temperature_celcius": 80, "producer_id": "producer